# Getting Started
**Make sure to clear the cell output before check into GitHub**
## Setup environment
If you don't have a .venv (Python 3.xx) Python kernel environment in the top right items of this window, then the first thing to do is setup the Python environment kernel:
- Go to View->Command Palette->Python: Create Environment... and run this command
- Select this Python kernel in top right of this window as your running environment

## Get Dataset
To training you need the dataset. The dataset for tennis ball can be found on Roboflow.
On Roboflow you can search for tennis dataset here: https://universe.roboflow.com/search

There are pretrained model with the dataset, so in some case if there is already a pretrained mdoel that is accurate enough for your use case, then you don't need to retrain the mdoel.]

The dataset that is used in this project is: https://universe.roboflow.com/tennistracker-dogbm/tennis-tracker-duufq/dataset/20#

We will use YOLO8 model, which is not the latest YOLO model, however it is small and efficient to turn on a mobile device. You can retain pretrained classes and add new specific classes by using a technique called partial fine-tuning or custom head training.

- Create a Roboflow account and log in
- Select "Download dataset" option
- Select "Show download code" option and accept the terms of agreement, you might need to go back to previous step if this is the first time you accept the agreement
- Copy the Jupyter code snipplet to below
- Replace the api_key with your API key from Roboflow under "Settings->API Keys"

Notes: When you run the first time, it might complained that you need a ipykernal environment as an error. Resolve the error by click on the Create Python Kernel environment. You can also go to View->Command Palette->Python: Create Environment...

In [ ]:
%pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="wNKC4X8SBzsq9OEpxrLC")
project = rf.workspace("tennistracker-dogbm").project("tennis-tracker-duufq")
version = project.version(20)
dataset = version.download("yolov8")
                

## Training
We will training using YOLO model for object detection. YOLO is used since it is for object detection and tracking from frame to frame. 

The dataset has 3 directories:
- train: this is the images used to train the model
- test: this is the images used to test the model after training is done
- valid: this is used during the training process to tune model hyperparameters and prevent overfitting

## Start training process
To train using Ultralytics API, which should be straight forward with the number of epochs and image size. This will probably not run on the laptop without Nvidia GPU.
Doc on Yolov5 https://docs.ultralytics.com/models/yolov5/#performance-metrics

There are many Yolo v8 models. The 2 opposite models are:
- yolov8x.pt where 'x' is extra large parameters which is used for high accuracy
- yolov8n.pt where 'n' is nano parametrs which is small and fast but lack accuracy. We will use this model since our use case is for the device

In [ ]:
%pip install ultralytics
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")  # Load the YOLOv5 nano model
print("data set location:", dataset.location)
# Train the model on the dataset
model.train(data=dataset.location + "/data.yaml", epochs=100, imgsz=640)


## Google Collab
This will probably not run on the laptop, so you should transfer the code to Google Collab to run on a GPU:
- Create and login account on Google Collab https://colab.research.google.com/
- Upload this notebook by drag-n-drop the file into the File->Open notebook dialog
- Change your runtime to T4 GPU under Runtime->Change runtime type
- Run all the start the training and output should be like:
```
  Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     12/100      14.5G      3.171      2.148     0.9224         30        640: 100%|██████████| 27/27 [00:35<00:00,  1.32s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.97it/s]                   all        100        101      0.548      0.297       0.23     0.0472

```
- After the run, the models need to be copied to your Google drive. The below code is only needed if you are running this in Google Collab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/runs/detect/train* /content/drive/MyDrive/my_models/